# Part 1: Setting up Elasticsearch

[Get started with Elasticsearch](https://www.elastic.co/guide/en/elasticsearch/reference/current/getting-started.html)
1. Download the Elasticsearch archive (Linux), set up a local server
2. Create a client connection to the local Elasticsearch instance

In [1]:
!wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.8.0-linux-x86_64.tar.gz -q
!tar -xzf elasticsearch-7.8.0-linux-x86_64.tar.gz
!chown -R daemon:daemon elasticsearch-7.8.0

In [2]:
import os
from subprocess import Popen, PIPE, STDOUT
es_process = Popen(['elasticsearch-7.8.0/bin/elasticsearch'], 
                  stdout=PIPE, stderr=STDOUT,
                  preexec_fn=lambda: os.setuid(1)
                 )

In [10]:
# Wait for a few minutes for the local host to start
#!curl -X GET "localhost:9200/"
!curl -X GET "localhost:9200/"

{
  "name" : "ed3170609ddf",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "5ACWjmXnRmGhLdRazrIQDw",
  "version" : {
    "number" : "7.8.0",
    "build_flavor" : "default",
    "build_type" : "tar",
    "build_hash" : "757314695644ea9a1dc2fecd26d1a43856725e65",
    "build_date" : "2020-06-14T19:35:50.234439Z",
    "build_snapshot" : false,
    "lucene_version" : "8.5.1",
    "minimum_wire_compatibility_version" : "6.8.0",
    "minimum_index_compatibility_version" : "6.0.0-beta1"
  },
  "tagline" : "You Know, for Search"
}


In [11]:
!pip install elasticsearch

In [12]:
# Python API documentation: https://elasticsearch-py.readthedocs.io/en/master/api.html
from elasticsearch import Elasticsearch
from elasticsearch import helpers

es_client = Elasticsearch(
    ['localhost']
)

In [13]:
es_client.info()

{'cluster_name': 'elasticsearch',
 'cluster_uuid': '5ACWjmXnRmGhLdRazrIQDw',
 'name': 'ed3170609ddf',
 'tagline': 'You Know, for Search',
 'version': {'build_date': '2020-06-14T19:35:50.234439Z',
  'build_flavor': 'default',
  'build_hash': '757314695644ea9a1dc2fecd26d1a43856725e65',
  'build_snapshot': False,
  'build_type': 'tar',
  'lucene_version': '8.5.1',
  'minimum_index_compatibility_version': '6.0.0-beta1',
  'minimum_wire_compatibility_version': '6.8.0',
  'number': '7.8.0'}}

#Part 2: Walking through an embedding-based retrieval system
As shown in the chart above, there are two main steps in the embedding-based retrieval system using Elasticsearch:
* Indexing: documents are first converted to vectors using deep learning models (aka embedding models). They are then indexed and stored on disk by Elasticsearch.
* Retrieving: a user query is first converted to its vector representation. Elasticsearch then uses this query vector to evaluate the similarity against indexed documents and returns top-scored ones.

In this example, we will use a universal sentence encoder model which has been trained to learn the representation of a sentence semantic meaning from large public corpus. Such models usually provide a decent baseline for NLP tasks. In practice it is necessary to train a deep model to learn the embedding for the target applications for performance boosting.

The code in this section is adapted from Elasticsearch's blog [text similarity search with vectors](https://www.elastic.co/blog/text-similarity-search-with-vectors-in-elasticsearch)

### Download MovieLens
There are 62,423 movies in this dataset (~250MB)
```python
	movieId	title	genres
  1	Toy Story (1995)	Adventure|Animation|Children|Comedy|Fantasy
  2	Jumanji (1995)	Adventure|Children|Fantasy
  3	Grumpier Old Men (1995)	Comedy|Romance
```

In [14]:
!wget http://files.grouplens.org/datasets/movielens/ml-25m.zip

--2020-08-26 18:45:44--  http://files.grouplens.org/datasets/movielens/ml-25m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261978986 (250M) [application/zip]
Saving to: ‘ml-25m.zip’

ml-25m.zip          100%[===================>] 249.84M  75.3MB/s    in 3.5s    

2020-08-26 18:45:47 (70.4 MB/s) - ‘ml-25m.zip’ saved [261978986/261978986]



In [15]:
!unzip ml-25m.zip

Archive:  ml-25m.zip
   creating: ml-25m/
  inflating: ml-25m/tags.csv         
  inflating: ml-25m/links.csv        
  inflating: ml-25m/README.txt       
  inflating: ml-25m/ratings.csv      
  inflating: ml-25m/genome-tags.csv  
  inflating: ml-25m/genome-scores.csv  
  inflating: ml-25m/movies.csv       


In [16]:
import pandas as pd
data = pd.read_csv('ml-25m/movies.csv').drop_duplicates()
data.head(3)

movieId  ...                                       genres
0        1  ...  Adventure|Animation|Children|Comedy|Fantasy
1        2  ...                   Adventure|Children|Fantasy
2        3  ...                               Comedy|Romance

[3 rows x 3 columns]

In [17]:
len(data)

62423

## Build index with document vectors
Embedding model: load pretrained sentence encoder model. More pretrained universal sentence encoder model variants are available [here](https://tfhub.dev/google/collections/universal-sentence-encoder/1)



Elasticsearch index API
([reference](https://www.elastic.co/guide/en/elasticsearch/reference/master/docs.html))
* single document: **Index** / Get / Delete / Update
* multi-document: Multi get / **Bulk** / Delete by query / Update by query / Reindex

```python
{
    '_index': 'index-name',
    '_id': 42,
    'title': 'Hello World!',
    'body': "..."
}
{
    '_op_type': 'delete',
    '_index': 'index-name',
    '_id': 42,
}
{
    '_op_type': 'update',
    '_index': 'index-name',
    '_id': 42,
    'doc': {'question': 'The life, universe and everything.'}
}
```

In [18]:
import tensorflow_hub as hub
from timeit import default_timer as timer
import json

In [30]:
# Requires tf 2.0; Use pre-training embedding vector.
# TWC: May need to use my own DL code
# TWC: Look at link: https://tfhub.dev/google/
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5")

In [29]:
sample_output = embed(["I like action movies"])
print(sample_output.shape) # (1, 512)
sample_output

(1, 512)


<tf.Tensor: shape=(1, 512), dtype=float32, numpy=
array([[ 4.54100966e-02,  1.16236845e-03,  3.70393023e-02,
         7.48189390e-02,  2.99990624e-02, -1.68460589e-02,
         3.42947897e-03,  6.02519996e-02, -3.68532315e-02,
        -6.27319217e-02,  2.08882149e-02,  2.56681573e-02,
        -4.20052409e-02, -2.68927701e-02, -8.85998365e-03,
         3.28758173e-02,  1.31798759e-01, -7.09365681e-03,
        -6.93769753e-02,  4.30495478e-03,  5.08708805e-02,
        -1.73859112e-02, -5.91872856e-02, -2.48534512e-02,
        -2.29507964e-02,  8.20938218e-03, -4.72139791e-02,
        -3.52629013e-02, -6.15934022e-02, -1.18936319e-02,
        -9.24651474e-02,  4.27385792e-02, -1.49404360e-02,
        -9.98184085e-02,  1.68038551e-02,  4.64366786e-02,
        -5.77947646e-02, -4.91680577e-02, -2.66500562e-03,
         2.96932142e-02,  2.74568293e-02,  1.63505170e-02,
         3.51132010e-03, -3.84898037e-02,  4.63070683e-02,
         7.64366537e-02, -2.45429166e-02,  2.92075109e-02,
      

In [23]:
# Constants
INDEX_NAME = "movie_title"
BATCH_SIZE = 200
SEARCH_SIZE = 10
# TWC: Mappings = schema 
MAPPINGS = {
    'mappings': {'_source': {'enabled': 'true'},
                 'dynamic': 'true',
                 'properties': {'title_vector':
                                {'dims': 512, 'type': 'dense_vector'},
                                'movie_id': {'type': 'keyword'},
                                'genres': {'type': 'keyword'},
                                'title': {'type': 'keyword'}
                                }
                 },
    'settings': {'number_of_replicas': 1, 'number_of_shards': 2}
}

In [24]:
def index_movie_lens(df, num_doc=500):
    print("creating the {} index.".format(INDEX_NAME))
    es_client.indices.delete(index=INDEX_NAME, ignore=[404])
    es_client.indices.create(index=INDEX_NAME, body=json.dumps(MAPPINGS)) #TWC: Load the schema

    requests = []
    count = 0
    start = timer()

    for row_index, doc in df.iterrows():

        # Specify the index size to avoid long waiting time
        if count >= num_doc:
            break

        # Construct requests
        if len(requests) < BATCH_SIZE:

            title_text = doc.title
            genres_text = doc.genres
            title_vector = embed([title_text]).numpy().tolist()[0]

            request = {
                "_op_type": "index",
                "_index": INDEX_NAME,
                "_id": row_index,
                "title": title_text,
                "genres": genres_text,
                "title_vector": title_vector,
                "movie_id": doc.movieId
            }

            requests.append(request)

        # Bulk helper: https://elasticsearch-py.readthedocs.io/en
        # /master/helpers.html#bulk-helpers
        else:
            helpers.bulk(es_client, requests)
            count += len(requests)
            requests.clear()
            if count % (BATCH_SIZE * 2) == 0:
                print("Indexed {} documents in {:.2f} seconds.".format(
                    count, timer()-start))

    # Index the remaining
    helpers.bulk(es_client, requests)
    end = timer()

    print("Done indexing {} documents in {:.2f} seconds.".format(
        count, end - start))


In [25]:
index_movie_lens(data, num_doc=2000)

creating the movie_title index.
Indexed 400 documents in 22.62 seconds.
Indexed 800 documents in 44.63 seconds.
Indexed 1200 documents in 65.80 seconds.
Indexed 1600 documents in 86.22 seconds.
Indexed 2000 documents in 106.32 seconds.
Done indexing 2000 documents in 106.33 seconds.


## Search with query vector

The raw text query is converted to `query_vector` and stored as a parameter. Elasticsearch provides a `cosineSimilarity` function in its native scripting language to compute the similarity scores to the query using a `script_score` query:

```python
{
  "script_score": {
    "query": {"match_all": {}},
    "script": {
      "source": "cosineSimilarity(params.query_vector, 'title_vector') + 1.0",
      "params": {"query_vector": query_vector}
    }
  }
}
```

In [26]:
def return_top_movies(query):

    embedding_start = timer()
    query_vector = embed([query]).numpy().tolist()[0] #TWC: Tensor -> NumPy -> pythonLis
    embedding_time = timer() - embedding_start
    formula = "cosineSimilarity(params.query_vector, 'title_vector') + 1.0"
    #TWC: ES requires score greater than 0
    
    script_query = {
        "script_score": {
            "query": {"match_all": {}},
            "script": {
                "source": formula,
                "params": {"query_vector": query_vector}
            }
        }
    }

    search_start = timer()
    response = es_client.search(
        index=INDEX_NAME,
        body={
            "size": SEARCH_SIZE,
            "query": script_query,
            "_source": {"includes": ["title", "genres"]}
        }
    )
    search_time = timer() - search_start

    print()
    print("{} total hits.".format(response["hits"]["total"]["value"])) #TWC: hits means the number of documents scanned.
    print("embedding time: {:.2f} ms".format(embedding_time * 1000))
    print("search time: {:.2f} ms".format(search_time * 1000))
    for hit in response["hits"]["hits"]:

        print("id: {}, score: {}".format(hit["_id"], hit["_score"] - 1))
        print(hit["_source"])
        print()


In [49]:
return_top_movies("Rommance")


2000 total hits.
embedding time: 89.22 ms
search time: 46.71 ms
id: 222, score: 0.3180373999999999
{'genres': 'Drama|Thriller', 'title': 'Disclosure (1994)'}

id: 1204, score: 0.25209269999999995
{'genres': 'Comedy|Drama|Romance', 'title': 'Trust (1990)'}

id: 451, score: 0.25098430000000005
{'genres': 'Crime|Drama|Thriller', 'title': 'Fresh (1994)'}

id: 611, score: 0.24703720000000007
{'genres': 'Comedy', 'title': 'Ed (1996)'}

id: 500, score: 0.23323209999999994
{'genres': 'Comedy', 'title': 'North (1994)'}

id: 38, score: 0.23098909999999995
{'genres': 'Comedy|Romance', 'title': 'Clueless (1995)'}

id: 599, score: 0.2225999999999999
{'genres': 'Drama', 'title': 'Century (1993)'}

id: 847, score: 0.2220576999999999
{'genres': 'Crime|Drama|Romance|Thriller', 'title': 'Bound (1996)'}

id: 249, score: 0.22020440000000008
{'genres': 'Comedy|Romance', 'title': 'I.Q. (1994)'}

id: 1322, score: 0.2201244
{'genres': 'Drama|Romance', 'title': 'Shine (1996)'}



# Part 3: Approximate nearest neighbor (ANN) algorithms

Elasticsearch's blog mentioned the current limition that vectors can be used for scoring documents, but not in the initial retrieval step. They also shared their [ongoing work](https://github.com/elastic/elasticsearch/issues/42326) about approximate neareast neighbor algorithm implementation to make the retrieval step scalable. In this section we'll go through a few common ANN algorithms with open-sourced library [`nmslib`](https://github.com/nmslib/nmslib) and [`faiss`](https://github.com/facebookresearch/faiss)
* Locality-sensitive hashing
* Product quantization with inverted file
* Hierarchical Navigable Small World Graphs

For additional ANN algorithms and performance comparisons, please refer this [ann-benchmarks](https://github.com/erikbern/ann-benchmarks).

### Locality-sensitive hashing (LSH)
LSH is a very classical binary hash. Its core is to create multiple hash functions to map vectors into binary codes. Vectors closely related are expected to hashed into the same codes.

### Product quantization with inverted file (IVFPQ)
Product quantization adopts k-means as its core quantizer and drastically increases the number of centroids by dividing each vector into many subvectors and runs the quantizer on all of these subvectors. The IVFPQ index relies on two levels of quantization. The main idea is to approximate a vector y as `y = q1(y) + q2(y-q1(y))` where q1 is a coarse quantizer (e.g. k-means clustering) and q2 is a fine quantizer (i.e., product quantizer)

### Hierarchical Navigable Small World Graphs (HNSW)
This method relies on exploring the graph based on the closeness relation between a node and its neighbors and neighbors’ neighbors and. HNSW stores the full length vectors and the full graph structure in memory (RAM).

In [35]:
!pip install faiss
!apt-get install libomp-dev
!pip install nmslib
import faiss
import nmslib

     |████████████████████████████████| 4.7MB 2.8MB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libomp5
Suggested packages:
  libomp-doc
The following NEW packages will be installed:
  libomp-dev libomp5
0 upgraded, 2 newly installed, 0 to remove and 35 not upgraded.
Need to get 239 kB of archives.
After this operation, 804 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libomp5 amd64 5.0.1-1 [234 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libomp-dev amd64 5.0.1-1 [5,088 B]
Fetched 239 kB in 1s (382 kB/s)
Selecting previously unselected package libomp5:amd64.
(Reading database ... 144556 files and directories currently installed.)
Preparing to unpack .../libomp5_5.0.1-1

In [50]:
documents = data['title'].to_list()[:2000]
# OOM for large document size
embeddings = embed(documents).numpy()

In [51]:
embeddings.shape

(2000, 512)

In [52]:
class DemoIndexLSH():
    def __init__(self, dimension, documents, embeddings):
        self.dimension = dimension
        self.documents = documents
        self.embeddings = embeddings

    def build(self, num_bits=8):
        self.index = faiss.IndexLSH(self.dimension, num_bits)
        self.index.add(self.embeddings)

    # For demo purpose, we expect 1 query at a time
    def query(self, input_embedding, k=5):
        distances, indices = self.index.search(input_embedding, k)

        return [(distance, self.documents[index]) for
                distance, index in zip(distances[0], indices[0])]

index_lsh = DemoIndexLSH(512, documents, embeddings)
index_lsh.build(num_bits=16)


In [53]:
# Ref: https://github.com/facebookresearch/faiss/wiki/Lower-memory-footprint
class DemoIndexIVFPQ():
    def __init__(self, dimension, documents, embeddings):
        self.dimension = dimension
        self.documents = documents
        self.embeddings = embeddings

    def build(self,
              number_of_partition=2,
              number_of_subquantizers=2,
              subvector_bits=4):
        quantizer = faiss.IndexFlatL2(self.dimension)
        self.index = faiss.IndexIVFPQ(quantizer,
                                      self.dimension,
                                      number_of_partition,
                                      number_of_subquantizers,
                                      subvector_bits)
        self.index.train(self.embeddings)
        self.index.add(self.embeddings)

    # For demo purpose, we expect 1 query at a time
    def query(self, input_embedding, k=5):
        distances, indices = self.index.search(input_embedding, k)

        return [(distance, self.documents[index]) for
                distance, index in zip(distances[0], indices[0])]

index_pq = DemoIndexIVFPQ(512, documents, embeddings)
index_pq.build()


In [54]:
# Ref: https://nmslib.github.io/nmslib/quickstart.html
class DemoHNSW():
    def __init__(self, dimension, documents, embeddings):
        self.dimension = dimension
        self.documents = documents
        self.embeddings = embeddings

    def build(self):
        self.index = nmslib.init(method='hnsw', space='cosinesimil')
        self.index.addDataPointBatch(self.embeddings)
        self.index.createIndex({'post': 2}, print_progress=True)

    def query(self, input_embedding, k=5):
        indices, distances = self.index.knnQuery(input_embedding, k)

        return [(distance, self.documents[index]) for
                distance, index in zip(distances, indices)]

index_hnsw = DemoHNSW(512, documents, embeddings)
index_hnsw.build()


In [55]:
# Baseline: exhaustive search
class DemoIndexFlatL2():
    def __init__(self, dimension, documents, embeddings):
        self.dimension = dimension
        self.documents = documents
        self.embeddings = embeddings

    def build(self):
        self.index = faiss.IndexFlatL2(self.dimension)
        self.index.add(self.embeddings)

    # For demo purpose, we expect 1 query at a time
    def query(self, input_embedding, k=5):
        distances, indices = self.index.search(input_embedding, k)

        return [(distance, self.documents[index]) for
                distance, index in zip(distances[0], indices[0])]

index_flat = DemoIndexFlatL2(512, documents, embeddings)
index_flat.build()


In [56]:
def return_ann_top_movies(ann_index, query, k=SEARCH_SIZE):
    query_vector = embed([query]).numpy()
    search_start = timer()
    top_docs = ann_index.query(query_vector, k)
    search_time = timer() - search_start
    print("search time: {:.2f} ms".format(search_time * 1000))
    return top_docs


In [57]:
return_ann_top_movies(index_flat, "romance")

search time: 1.05 ms


[(0.95573366, 'True Romance (1993)'),
 (1.2160163, 'Love Serenade (1996)'),
 (1.2626679, 'Love Affair (1994)'),
 (1.3447753, 'Kissed (1996)'),
 (1.3752131, 'In Love and War (1996)'),
 (1.3804029, 'Casablanca (1942)'),
 (1.3832319, 'Flirt (1995)'),
 (1.38626, 'Moonlight and Valentino (1995)'),
 (1.3862813, 'Hotel de Love (1996)'),
 (1.3907104, 'Intimate Relations (1996)')]

In [58]:
return_ann_top_movies(index_lsh, "romance")

search time: 0.19 ms


[(2.0, 'Visitors, The (Visiteurs, Les) (1993)'),
 (2.0, 'City Hall (1996)'),
 (2.0, 'Paradise Road (1997)'),
 (3.0, 'When a Man Loves a Woman (1994)'),
 (3.0, 'Cosi (1996)'),
 (3.0, 'Haunted World of Edward D. Wood Jr., The (1996)'),
 (3.0, 'Eddie (1996)'),
 (3.0, 'Ransom (1996)'),
 (3.0, 'Time to Kill, A (1996)'),
 (3.0, 'Mirage (1995)')]

In [59]:
return_ann_top_movies(index_pq, "romance")

search time: 0.54 ms


[(1.07124, 'Streetcar Named Desire, A (1951)'),
 (1.07124, 'Moonlight Murder (1936)'),
 (1.0847104, 'To Kill a Mockingbird (1962)'),
 (1.0847104, 'Meet John Doe (1941)'),
 (1.0867723, 'Moonlight and Valentino (1995)'),
 (1.0901785, 'Laura (1944)'),
 (1.0901785, 'Rebecca (1940)'),
 (1.0901785, 'African Queen, The (1951)'),
 (1.0901785, 'Gigi (1958)'),
 (1.0901785, 'Scarlet Letter, The (1926)')]

In [46]:
return_ann_top_movies(index_hnsw, "romance")

search time: 0.34 ms


[(0.47786665, 'True Romance (1993)'),
 (0.6080081, 'Love Serenade (1996)'),
 (0.63133395, 'Love Affair (1994)'),
 (0.6723877, 'Kissed (1996)'),
 (0.6876065, 'In Love and War (1996)'),
 (0.6916158, 'Flirt (1995)'),
 (0.69312984, 'Moonlight and Valentino (1995)'),
 (0.69314075, 'Hotel de Love (1996)'),
 (0.69535506, 'Intimate Relations (1996)'),
 (0.6985383, 'Love in Bloom (1935)')]

# References

- [MovieLens 25M Dataset](https://grouplens.org/datasets/movielens/25m/)
- [Text Similarity Search in Elasticsearch](https://www.elastic.co/blog/text-similarity-search-with-vectors-in-elasticsearch)
- [Benchmarks of approximate nearest neighbor libraries in Python](https://github.com/erikbern/ann-benchmarks)